In [ ]:
import networkx as nx
from collections import defaultdict

In [ ]:
with open("input.txt", "r") as f:
    s = f.read()

In [ ]:
from IPython.display import Image

def draw(G, prog='dot', args=''):
    """Helper for plotting a NetworkX DiGraph"""
    A = nx.nx_agraph.to_agraph(G)
    
    # put all root nodes at the top
    roots = [n for n, d in G.in_degree() if d == 0]
    s = A.add_subgraph(rank='same')
    s.add_nodes_from(roots)

    A.layout(prog=prog, args=args)
    return Image(A.draw(format='png'))

In [ ]:
G = nx.DiGraph()
current_node = "file_system"
G.add_node(current_node, type="foo")
G.add_node("/", type="dir")
G.add_edge(current_node, "/")
for row in s.split("\n"):
    print(row)
    if row.startswith("$ "):
        command = row.replace("$ ", "")
        if command.startswith("cd "):
            to_loc = command.replace("cd ", "")
            if to_loc == "..": 
                preds = set(G.predecessors(current_node))
                assert len(preds) == 1, f"{current_node} has more than one predecessors {preds}!"
                current_node = preds.pop()
            else:
                succs = set(G.successors(current_node))
                to_loc_match = {
                    succ 
                    for succ in succs 
                    if succ.startswith(to_loc) and nx.get_node_attributes(G, "type")[succ] == "dir"
                }
                assert len(to_loc_match) == 1, f"{current_node} has more than one matched successors {to_loc_match}!"
                current_node = to_loc_match.pop()
        elif command == "ls":
            pass
    elif row.startswith("dir "):
        dir = row.replace("dir ", "")
        if (dir in G.nodes):
            dir = f"{dir}_{len([str(n) for n in G.nodes if str(n).split('_')[0] == dir])}"
            print(dir)
        print(f"  {current_node}: {dir}")
        G.add_node(dir, type="dir")
        G.add_edge(current_node, dir)
    else:
        size, file = row.split(" ")
        if (file in G.nodes):
            file = f"{file}_{len([str(n) for n in G.nodes if str(n).split('_')[0] == file])}"
            print(file)
        G.add_node(file, type="file", size=int(size))
        G.add_edge(current_node, file)


In [ ]:
draw(G)

In [ ]:
dirs = {node for node in G.nodes if nx.get_node_attributes(G, "type")[node] == "dir"}
dirs

In [ ]:
dirs_size = defaultdict(int)

In [ ]:
def sum_size(parent_dir):
    for succ in G.successors(parent_dir):
        if nx.get_node_attributes(G, "type")[succ] == "dir":
            sum_size(succ)
        elif nx.get_node_attributes(G, "type")[succ] == "file":
            ancs = nx.ancestors(G, succ)
            for anc in ancs:
                dirs_size[anc] += nx.get_node_attributes(G, "size")[succ]

In [ ]:
sum_size("/")
dirs_size

In [ ]:
sum([size for size in dirs_size.values() if size<100000])

In [ ]:
target_size = dirs_size["/"] - (70000000 - 30000000)
target_size

In [ ]:
extra_size_after_delete = {dir: size-target_size for dir, size in dirs_size.items() if size>target_size}
extra_size_after_delete

In [ ]:
sorted(extra_size_after_delete.items(), key=lambda item: item[1])

In [ ]:
dirs_size["mdclfbs"]